## Benchmark Cellpose3D on Ovules / ATAS / A.T. atlas datasets

In [10]:
import os
import gc
import sys
import h5py
import tifffile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import matplotlib.font_manager
from matplotlib import rcParams
from IPython.display import display

font_list = []
fpaths = matplotlib.font_manager.findSystemFonts()
for i in fpaths:
    try:
        f = matplotlib.font_manager.get_font(i)
        font_list.append(f.family_name)
    except RuntimeError:
        pass

font_list = set(font_list)
plot_font = 'Helvetica' if 'Helvetica' in font_list else 'FreeSans'

rcParams['font.family'] = plot_font
rcParams.update({'font.size': 10})

In [12]:
sys.path.append('../')

from cellstitch import evaluation
from utils import h5_converter, csv_generator

### Prepare image dataset

**Ovules**

In [4]:
ovules_path = '../data/ovules/test/'
ovules_output_path = 'data/ovules'
ovules_res_path = 'results/ovules'

# if not os.path.exists(ovules_output_path):
#     os.makedirs(ovules_output_path, exist_ok=True)

fnames = [
    f.rpartition('.')[0]
    for f in sorted(os.listdir(ovules_path))
]

imgs = [
    h5py.File(os.path.join(ovules_path, f))['raw'][:]
    for f in sorted(os.listdir(ovules_path))
    if f[-2:] == 'h5'
]

masks = [
    h5py.File(os.path.join(ovules_path, f))['label'][:]
    for f in sorted(os.listdir(ovules_path))
]

# for (f, img) in zip(fnames, imgs):
#     tifffile.imwrite(os.path.join(ovules_output_path, f+'.tif'), img)
    
del ovules_path

In [ ]:
# Prepare output h5 file for Cellpose3D
h5_converter.prepare_images(data_path=ovules_output_path)

[11:48:52] Processing image 1/7 in ""
[11:49:16] Processing image 2/7 in ""
[11:49:28] Processing image 3/7 in ""
[11:49:33] Processing image 4/7 in ""
[11:49:52] Processing image 5/7 in ""
[11:49:57] Processing image 6/7 in ""
[11:50:12] Processing image 7/7 in ""


In [9]:
# Generate filename list .csv
with open('files_test.csv', 'w') as ofile:
    for filename in fnames:
        ofile.write(filename+'.tif;\n')
        

**Calculate metrics**

In [21]:
metrics = np.zeros((len(fnames), 7))

for i, fname in enumerate(fnames):
    print('Calculating metrics for {}...'.format(fname))
    y_pred = tifffile.imread(os.path.join(ovules_res_path, 'instances_' + fname+'.tif')).astype(np.int64)
    y_true = masks[i]
    
    assert y_pred.shape == y_true.shape, "Inconsistent shapes btw ground-truth & predictions"
    
    ap25 = evaluation.average_precision(y_true, y_pred, 0.25)[0]
    ap75 = evaluation.average_precision(y_true, y_pred, 0.75)[0]
    ap50, tp, fp, fn = evaluation.average_precision(y_true, y_pred, 0.5)
    mAP = np.mean([ap25, ap50, ap75])
    prec, recall, f1 = tp/(tp+fp), tp/(tp+fn), tp/(tp+0.5*(fp+fn))
    
    metrics[i] = [prec, recall, f1, ap25, ap50, ap75, mAP]
    gc.collect()
    
del y_true, y_pred, ap25, ap50, ap75, tp, fp, fn, prec, recall, f1


Calculating metrics for N_294_final_crop_ds2...
Calculating metrics for N_435_final_crop_ds2...
Calculating metrics for N_441_final_crop_ds2...
Calculating metrics for N_511_final_crop_ds2...
Calculating metrics for N_522_final_crop_ds2...
Calculating metrics for N_590_final_crop_ds2...
Calculating metrics for N_593_final_crop_ds2...


In [22]:
metrics_df = pd.DataFrame(metrics, index=fnames, columns=['prec', 'recall', 'f1', 'AP25', 'AP50', 'AP75', 'mAP'])
metrics_df.head()

,prec,recall,f1,AP25,AP50,AP75,mAP
N_294_final_crop_ds2,0.787058,0.890789,0.835717,0.772390,0.717795,0.627064,0.705750
N_435_final_crop_ds2,0.683903,0.956934,0.797702,0.758435,0.663482,0.566736,0.662884
N_441_final_crop_ds2,0.509467,0.946085,0.662291,0.555937,0.495093,0.439647,0.496892
N_511_final_crop_ds2,0.695815,0.933801,0.797430,0.740741,0.663105,0.597578,0.667142
N_522_final_crop_ds2,0.593947,0.962136,0.734482,0.638360,0.580381,0.522987,0.580576


In [24]:
metrics_df.mean(0)

prec      0.681356
recall    0.938937
f1        0.785273
AP25      0.712946
AP50      0.650368
AP75      0.572656
mAP       0.645323
dtype: float64

In [6]:
metrics_df.std(0)

prec      0.098321
recall    0.025166
f1        0.065394
AP25      0.083175
AP50      0.084757
AP75      0.072629
mAP       0.079221
dtype: float64

In [23]:
metrics_df.to_csv('results/ovules_cp3d_variant.csv', index=True)

**ATAS**

In [13]:
%ls ../data/ATAS/raw/ | head -n 5

0hrs_plant13_trim-acylYFP.npy
0hrs_plant15_trim-acylYFP.npy
0hrs_plant18_trim-acylYFP.npy
0hrs_plant1_trim-acylYFP.npy
0hrs_plant2_trim-acylYFP.npy


In [13]:
atas_path = '../data/ATAS/raw/'
mask_path = '../data/ATAS/labels/'
atas_output_path = 'data/ATAS/'

if not os.path.exists(atas_output_path):
    os.makedirs(atas_output_path, exist_ok=True)

fnames = [
    f.rpartition('.')[0]
    for f in sorted(os.listdir(atas_path))
]

imgs = [
    np.load(os.path.join(atas_path, f))
    for f in sorted(os.listdir(atas_path))
    if f[-3:] == 'npy'
]

# for (f, img) in zip(fnames, imgs):
#     tifffile.imwrite(os.path.join(atas_output_path, f+'.tif'), img)
    
del atas_path

In [8]:
# Prepare output h5 file for Cellpose3D
h5_converter.prepare_images(data_path=atas_output_path)

[02:02:17] Processing image 1/125 in ""
[02:02:17] Processing image 2/125 in ""
[02:02:18] Processing image 3/125 in ""
[02:02:18] Processing image 4/125 in ""
[02:02:18] Processing image 5/125 in ""
[02:02:18] Processing image 6/125 in ""
[02:02:19] Processing image 7/125 in ""
[02:02:19] Processing image 8/125 in ""
[02:02:19] Processing image 9/125 in ""
[02:02:20] Processing image 10/125 in ""
[02:02:20] Processing image 11/125 in ""
[02:02:20] Processing image 12/125 in ""
[02:02:20] Processing image 13/125 in ""
[02:02:21] Processing image 14/125 in ""
[02:02:21] Processing image 15/125 in ""
[02:02:21] Processing image 16/125 in ""
[02:02:22] Processing image 17/125 in ""
[02:02:22] Processing image 18/125 in ""
[02:02:22] Processing image 19/125 in ""
[02:02:23] Processing image 20/125 in ""
[02:02:23] Processing image 21/125 in ""
[02:02:23] Processing image 22/125 in ""
[02:02:23] Processing image 23/125 in ""
[02:02:24] Processing image 24/125 in ""
[02:02:24] Processing ima

In [7]:
# Generate filename list .csv
with open('ATAS_files_test.csv', 'w') as ofile:
    for filename in fnames:
        ofile.write(os.path.join(atas_output_path, filename+'.h5;\n'))
        

**Calculate metrics**

In [24]:
# Load test set filenames


res_path = 'results/ATAS/'
test_fnames = [
    f.split('.')[0]
    for f in sorted(os.listdir('../results/ATAS/cellpose2d/'))
    if f[-3:] == 'npy'
]

output_fnames = [
    f.strip('instances_').split('.')[0]
    for f in sorted(os.listdir(res_path))
    if f.startswith('instance') and f[-3:] == 'tif'
]

fnames = np.intersect1d(test_fnames, output_fnames)  # filter out failed-to-segmented names

del test_fnames, output_fnames


In [ ]:
metrics = np.zeros((len(fnames), 7))

for i, fname in enumerate(fnames):
    print('Calculating metrics for {}...'.format(fname))
    y_pred = tifffile.imread(os.path.join(res_path, 'instances_' + fname+'.tif')).astype(np.int64)
    y_true = np.load(os.path.join(mask_path, fname+'.npy')).astype(np.int64)
    
    print(y_pred.shape, y_true.shape)
    
    assert y_pred.shape == y_true.shape, "Inconsistent shapes btw ground-truth & predictions"
    
    ap25 = evaluation.average_precision(y_true, y_pred, 0.25)[0]
    ap75 = evaluation.average_precision(y_true, y_pred, 0.75)[0]
    ap50, tp, fp, fn = evaluation.average_precision(y_true, y_pred, 0.5)
    mAP = np.mean([ap25, ap50, ap75])
    prec, recall, f1 = tp/(tp+fp), tp/(tp+fn), tp/(tp+0.5*(fp+fn))
    
    metrics[i] = [prec, recall, f1, ap25, ap50, ap75, mAP]
    gc.collect()
    
del y_true, y_pred, ap25, ap50, ap75, tp, fp, fn, prec, recall, f1


Calculating metrics for 0hrs_plant2_trim-acylYFP...
(22, 512, 512) (22, 512, 512)
Calculating metrics for 12hrs_plant18_trim-acylYFP...
(15, 512, 512) (15, 512, 512)
Calculating metrics for 12hrs_plant2_trim-acylYFP...
(13, 512, 512) (13, 512, 512)
Calculating metrics for 20hrs_plant13_trim-acylYFP...
(18, 512, 512) (18, 512, 512)
Calculating metrics for 20hrs_plant2_trim-acylYFP...
(18, 512, 512) (18, 512, 512)
Calculating metrics for 24hrs_plant13_trim-acylYFP...
(20, 512, 512) (20, 512, 512)
Calculating metrics for 24hrs_plant15_trim-acylYFP...
(18, 512, 512) (18, 512, 512)
Calculating metrics for 28hrs_plant1_trim-acylYFP...
(22, 512, 512) (22, 512, 512)
Calculating metrics for 32hrs_plant13_trim-acylYFP...
(21, 512, 512) (21, 512, 512)
Calculating metrics for 36hrs_plant15_trim-acylYFP...
(23, 512, 512) (23, 512, 512)
Calculating metrics for 36hrs_plant2_trim-acylYFP...
(24, 512, 512) (24, 512, 512)
Calculating metrics for 40hrs_plant13_trim-acylYFP...
(21, 512, 512) (21, 512, 512

In [41]:
sample_pred = tifffile.imread(os.path.join(res_path, 'instances_' + '8hrs_plant2_trim-acylYFP.tif')).astype(np.int64)
sample_mask = np.load(os.path.join(mask_path, '8hrs_plant2_trim-acylYFP.npy')).astype(np.int64)

In [33]:
metrics_df = pd.DataFrame(metrics, columns=['prec', 'recall', 'f1', 'AP25', 'AP50', 'AP75', 'mAP'])

print('Mean:')
print(metrics_df.mean(0))
print('\n\nStd')
print(metrics_df.std(0))

Mean:
prec      0.086227
recall    0.033811
f1        0.047978
AP25      0.243113
AP50      0.024850
AP75      0.000239
mAP       0.089401
dtype: float64


Std
prec      0.047915
recall    0.024572
f1        0.031966
AP25      0.078779
AP50      0.017091
AP75      0.000858
mAP       0.030225
dtype: float64


In [48]:
metrics_df.to_csv('results/atas_cp3d_variants.csv', index=True)

**Plant Atlas**

In [22]:
%ls ../data/plant_atlas/

Anther/        Filament_masks/  Pedicel/        Sepal_masks/
Anther_masks/  Leaf/            Pedicel_masks/  Valve/
Filament/      Leaf_masks/      Sepal/          Valve_masks/


In [4]:
# Directly copy .tif files to target directory
regions = ['Anther', 'Filament', 'Leaf', 'Pedicel', 'Sepal', 'Valve']

for region in regions:
    region_path = os.path.join('data', region)
    if not os.path.exists(region_path):
        os.makedirs(region_path)

        
del region, region_path

In [6]:
# Prepare output h5 file for Cellpose3D
for region in regions:
    print('Generating test files for {}...'.format(region))s
    region_path = os.path.join('data', region)
    h5_converter.prepare_images(data_path=region_path)

Generating test files for Anther...
[14:49:30] Processing image 1/30 in ""
[14:49:30] Processing image 2/30 in ""
[14:49:30] Processing image 3/30 in ""
[14:49:30] Processing image 4/30 in ""
[14:49:30] Processing image 5/30 in ""
[14:49:30] Processing image 6/30 in ""
[14:49:30] Processing image 7/30 in ""
[14:49:30] Processing image 8/30 in ""
[14:49:30] Processing image 9/30 in ""
[14:49:30] Processing image 10/30 in ""
[14:49:30] Processing image 11/30 in ""
[14:49:30] Processing image 12/30 in ""
[14:49:30] Processing image 13/30 in ""
[14:49:30] Processing image 14/30 in ""
[14:49:31] Processing image 15/30 in ""
[14:49:31] Processing image 16/30 in ""
[14:49:31] Processing image 17/30 in ""
[14:49:31] Processing image 18/30 in ""
[14:49:31] Processing image 19/30 in ""
[14:49:31] Processing image 20/30 in ""
[14:49:31] Processing image 21/30 in ""
[14:49:31] Processing image 22/30 in ""
[14:49:31] Processing image 23/30 in ""
[14:49:31] Processing image 24/30 in ""
[14:49:31] Pr

In [ ]:
# # Generate filename list .csv
# with open('files_test.csv', 'w') as ofile:
#     for filename in fnames:
# ofile.write(filename+'.tif;\n')

In [10]:
# Generate filename list .csv
for region in regions:
    region_path = os.path.join('data', region)
    fnames = [
        f.split('.')[0]
        for f in sorted(os.listdir(region_path))
        if f[-2:] == 'h5'
    ]
    
    with open(region+'_files_test.csv', 'w') as ofile:
        for filename in fnames:
            ofile.write(os.path.join(region_path, filename+'.h5;\n'))
            
del region, region_path, fnames

**Calculate metrics**

In [12]:
%ls ../data/plant_atlas/Anther_masks/

Anther_20.tif  Anther_26.tif  Anther_32.tif  Anther_38.tif  Anther_44.tif
Anther_21.tif  Anther_27.tif  Anther_33.tif  Anther_39.tif  Anther_45.tif
Anther_22.tif  Anther_28.tif  Anther_34.tif  Anther_40.tif  Anther_46.tif
Anther_23.tif  Anther_29.tif  Anther_35.tif  Anther_41.tif  Anther_47.tif
Anther_24.tif  Anther_30.tif  Anther_36.tif  Anther_42.tif  Anther_48.tif
Anther_25.tif  Anther_31.tif  Anther_37.tif  Anther_43.tif  Anther_49.tif


In [5]:
metrics_list = []

for region in regions:
    mask_path = os.path.join('../data/plant_atlas/{}_masks'.format(region))
    res_path = os.path.join('results', region)
    fnames = [
        f for f in sorted(os.listdir(mask_path))
        if f[-3:] == 'tif'
    ]
    
    metrics = np.zeros((len(fnames), 7))

    for i, fname in enumerate(fnames):
        print('Calculating metrics for {}...'.format(fname))
        y_pred = tifffile.imread(os.path.join(res_path, 'instances_' + fname)).astype(np.int64)
        y_true = tifffile.imread(os.path.join(mask_path, fname)).astype(np.int64)

        assert y_pred.shape == y_true.shape, "Inconsistent shapes btw ground-truth & predictions"

        ap25 = evaluation.average_precision(y_true, y_pred, 0.25)[0]
        ap75 = evaluation.average_precision(y_true, y_pred, 0.75)[0]
        ap50, tp, fp, fn = evaluation.average_precision(y_true, y_pred, 0.5)
        mAP = np.mean([ap25, ap50, ap75])
        prec, recall, f1 = tp/(tp+fp), tp/(tp+fn), tp/(tp+0.5*(fp+fn))

        metrics[i] = [prec, recall, f1, ap25, ap50, ap75, mAP]
        gc.collect()
        
    metrics_df = pd.DataFrame(metrics, index=fnames, columns=['prec', 'recall', 'f1', 'AP25', 'AP50', 'AP75', 'mAP'])
    metrics_list.append(metrics_df)
    
del y_true, y_pred, ap25, ap50, ap75, tp, fp, fn, prec, recall, f1, fnames, mask_path


Calculating metrics for Anther_20.tif...
Calculating metrics for Anther_21.tif...
Calculating metrics for Anther_22.tif...
Calculating metrics for Anther_23.tif...
Calculating metrics for Anther_24.tif...
Calculating metrics for Anther_25.tif...
Calculating metrics for Anther_26.tif...
Calculating metrics for Anther_27.tif...
Calculating metrics for Anther_28.tif...
Calculating metrics for Anther_29.tif...
Calculating metrics for Anther_30.tif...
Calculating metrics for Anther_31.tif...
Calculating metrics for Anther_32.tif...
Calculating metrics for Anther_33.tif...
Calculating metrics for Anther_34.tif...
Calculating metrics for Anther_35.tif...
Calculating metrics for Anther_36.tif...
Calculating metrics for Anther_37.tif...
Calculating metrics for Anther_38.tif...
Calculating metrics for Anther_39.tif...
Calculating metrics for Anther_40.tif...
Calculating metrics for Anther_41.tif...
Calculating metrics for Anther_42.tif...
Calculating metrics for Anther_43.tif...
Calculating metr

In [14]:
for region, metrics_df in zip(regions, metrics_list):
    print('Metrics for {}:\n'.format(region))
    print('Mean:')
    display(metrics_df.mean(0))
    print('\nStd:')
    display(metrics_df.std(0))
    print('\n\n')
    

Metrics for Anther:

Mean:


prec      0.452490
recall    0.544998
f1        0.487597
AP25      0.510258
AP50      0.327520
AP75      0.168433
mAP       0.335404
dtype: float64


Std:


prec      0.112868
recall    0.106887
f1        0.095653
AP25      0.060444
AP50      0.083998
AP75      0.078257
mAP       0.065507
dtype: float64




Metrics for Filament:

Mean:


...

prec      0.119064
recall    0.089202
f1        0.080725
AP25      0.085603
AP50      0.073880
AP75      0.053657
mAP       0.058035
dtype: float64




Metrics for Valve:

Mean:


prec      0.649826
recall    0.533767
f1        0.584930
AP25      0.600469
AP50      0.415087
AP75      0.211262
mAP       0.408939
dtype: float64


Std:


prec      0.063553
recall    0.051174
f1        0.050396
AP25      0.057598
AP50      0.050286
AP75      0.048633
mAP       0.044733
dtype: float64

In [8]:
for i, region in enumerate(regions):
    metrics_list[i].to_csv('results/{}_cp3d_variant.csv'.format(region), index=True)

---